In [10]:
# Ensure project root (OpenCEP) is on sys.path for imports like `from base.Pattern import Pattern`
import sys, os, pathlib
nb_dir = pathlib.Path(__file__).parent if "__file__" in globals() else pathlib.Path.cwd()
project_root = str((nb_dir / "..").resolve())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# OpenCEP Demo
This notebook demonstrates a simple ascending peak price pattern over Google stock updates.

In [11]:
from datetime import timedelta
from CEP import CEP
from base.Pattern import Pattern
from base.PatternStructure import SeqOperator, PrimitiveEventStructure
from condition.CompositeCondition import AndCondition
from condition.BaseRelationCondition import SmallerThanCondition
from condition.Condition import Variable, SimpleCondition
from stream.FileStream import FileInputStream, FileOutputStream
from plugin.stocks.Stocks import MetastockDataFormatter
import test

This pattern is looking for a short ascend in the Google peak prices:
```
PATTERN SEQ(GoogleStockPriceUpdate a, GoogleStockPriceUpdate b, GoogleStockPriceUpdate c)
WHERE a.PeakPrice < b.PeakPrice AND b.PeakPrice < c.PeakPrice
WITHIN 3 minutes
```

In [5]:
googleAscendPattern = Pattern(
        SeqOperator(PrimitiveEventStructure("GOOG", "a"), 
                    PrimitiveEventStructure("GOOG", "b"), 
                    PrimitiveEventStructure("GOOG", "c")),
        AndCondition(
            SmallerThanCondition(Variable("a", lambda x: x["Peak Price"]), 
                                 Variable("b", lambda x: x["Peak Price"])),
            SmallerThanCondition(Variable("b", lambda x: x["Peak Price"]), 
                                 Variable("c", lambda x: x["Peak Price"]))
        ),
        timedelta(minutes=3)
    )

Another way to define the above example is to use SimpleCondition and a lambda function:

In [6]:
googleAscendPattern = Pattern(
        SeqOperator(PrimitiveEventStructure("GOOG", "a"), 
                    PrimitiveEventStructure("GOOG", "b"), 
                    PrimitiveEventStructure("GOOG", "c")),
        SimpleCondition(Variable("a", lambda x: x["Peak Price"]), 
                        Variable("b", lambda x: x["Peak Price"]),
                        Variable("c", lambda x: x["Peak Price"]),
                        relation_op=lambda x,y,z: x < y < z),
        timedelta(minutes=3)
    )

Creating a CEP object for monitoring the first pattern from the example above:

In [7]:
cep = CEP([googleAscendPattern])

Defining a new file-based event stream formatted according to Metastock 7 format:

In [8]:
events = FileInputStream("../test/EventFiles/NASDAQ_SHORT.txt")

Applying an existing CEP object on an event stream created above and storing the resulting pattern matches to a file:

In [9]:
cep.run(events, FileOutputStream('../test/demo/Matches', 'output.txt'), MetastockDataFormatter())

0.009082